### 开包即用的MNIST手写数字识别

在本workshop，我将演示如何使用两个Canned Estimators（这些封装了迄今为止我们已经看到的较低级别的TensorFlow代码，并且使用了一个 [scikit-learn](scikit-learn.org)的API。 Canned Estimators有几个优点。

* 则无需管理会话Session，也无需为TensorBoard编写自己的逻辑，也无需保存和加载检查点。


* 您将获得开箱即用的分布式培训（当然，您将不得不小心有效地读取数据并设置群集）。

我们将使用输入函数 [input functions](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn)读取数据

* 这些为您提供批处理和其他功能，因此您不必亲自编写代码。


* 在结构化的 data nootbook 中，我们将使用新的 [Dataset API](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/docs_src/programmers_guide/datasets.md) - 一种高效读取和预处理大型数据集的有效方法

虽然我们在这里使用的 Estimators 是相对简单的（一个LinearClassifier和一个完全连接的深度神经网络），你可以在官网找到更有趣的 (including for [TensorFlow Wide and Deep](https://www.tensorflow.org/tutorials/wide_and_deep).

Estimators 有着相当多的输出

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 我们将使用Keras（包含在TensorFlow中）导入数据
(x_train, y_train), (x_test, y_test) = tf.contrib.keras.datasets.mnist.load_data()
# f = np.load('data/mnist.npz')  
# x_train, y_train = f['x_train'], f['y_train']  
# x_test, y_test = f['x_test'], f['y_test']  
# f.close()  

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Normalize the color values to 0-1
# (as imported, they're 0-255)
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


这是我们的输入函数:

* 设置 ```num_epochs``` 为 ```None``` 完全过一次训练样本

* 设置 ```batch_size``` 为 ```128```  每次训练 128个样本

* iteration：表示1次迭代（也叫training step），每次迭代更新1次网络结构的参数；

* batch-size：1次迭代所使用的样本量；

* epoch：1个epoch表示过了1遍训练集中的所有样本。


In [5]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, 
    shuffle=True  #随机打乱输入
)

In [6]:
test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # 遍历测试数据集一次
    shuffle=False # 不要对测试数据进行混淆
)

In [7]:
# 定义我们模型的特征
# 名称必须和我们在 train_input 中的一致
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

这里我们使用 ```LinearClassifier``` 分类器，这与softmax的作用是相同的

In [8]:
estimator = tf.estimator.LinearClassifier(feature_spec, 
                                          n_classes=10,
                                          model_dir="./graphs/canned/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_keep_checkpoint_max': 5, '_model_dir': './graphs/canned/linear', '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_task_id': 0, '_log_step_count_steps': 100, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_checkpoints_secs': 600, '_master': '', '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BC119FE860>, '_save_summary_steps': 100, '_task_type': 'worker'}


In [9]:
# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/canned/linear\model.ckpt.
INFO:tensorflow:loss = 294.73087, step = 1
INFO:tensorflow:global_step/sec: 558.709
INFO:tensorflow:loss = 67.24473, step = 101 (0.180 sec)
INFO:tensorflow:global_step/sec: 560.295
INFO:tensorflow:loss = 36.13924, step = 201 (0.178 sec)
INFO:tensorflow:global_step/sec: 583.241
INFO:tensorflow:loss = 67.71013, step = 301 (0.173 sec)
INFO:tensorflow:global_step/sec: 563.469
INFO:tensorflow:loss = 39.67474, step = 401 (0.175 sec)
INFO:tensorflow:global_step/sec: 608.135
INFO:tensorflow:loss = 35.52318, step = 501 (0.164 sec)
INFO:tensorflow:global_step/sec: 600.806
INFO:tensorflow:loss = 50.88822, step = 601 (0.167 sec)
INFO:tensorflow:global_step/sec: 533.338
INFO:tensorflow:loss = 44.3544, step = 701 (0.187 sec)
INFO:tensorflow:global_step/sec: 569.908
INFO:tensorflow:loss = 44.950783, step = 801 (0.176 sec)
INFO:tensorflow:global_step/sec: 641.376
INFO:tensorflow

In [10]:
# 应该只有90%的准确度
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2018-03-23-07:46:52
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-03-23-07:46:52
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9249, average_loss = 0.27294406, global_step = 1000, loss = 34.54988
{'loss': 34.54988, 'accuracy': 0.9249, 'global_step': 1000, 'average_loss': 0.27294406}


我们使用训练好的数据进行预测

In [ ]:
MAX_TO_PRINT = 5

# 这会返回一个生成器对象
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

这里我们使用DNN分类器(深度神经网络)全连接

In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], 
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

In [ ]:
# 这里我们训练2000步
estimator.train(train_input, steps=2000)

In [ ]:
# 准确度大概97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

可以打开tensorBoard查看模型

``` $ tensorboard --logdir=graphs/canned/ ```